# Introduction

Let's predict the cost of homes in parts of California using Python, specificilly the Scikit-learn library. Nine Feature variables and one Target variable will be utlized. Linear Regresson, Random Forest Regressor, and XGBoost will be used in Model Training. Accuracy, R

## Target Variable
medianHouseValue: Median house value for households within a block (measured in US Dollars).

## Feature Variables
longitude: A measure of how far west a house is (a higher value is farther west).
latitude: A measure of how far north a house is (a higher value is farther north).
housingMedianAge: Median age of a house within a block (a lower number implies a newer building).
totalRooms: Total number of rooms within a block.
totalBedrooms: Total number of bedrooms within a block.
population: Total number of people residing within a block.
households: Total number of households. where a household unit is comprised of a group of people for some block.
medianIncome: Median income for households within a block of houses (measured in tens of thousands of US Dollars).
oceanProximity: Location of the house in proximity to the ocean.

## Background
Scikit-learn is a machine learning library for the Python. It features various classification, regression, and cluster learning algorithms.

## Dependencies
pandas - To work with solid data-structures, n-dimensional matrices and perform exploratory data analysis.
matplotlib - To visualize data using 2D plots.
seaborn - To make 2D plots look pretty and readable.
scikit-learn - To create machine learning models easily and make predictions.
numpy - To work with arrays.

# Obtain & Load Data

In [ ]:
# Load packages
%pip install pandas
%pip install matplotlib
%pip install seaborn
%pip install scikit-learn
%pip install numpy
%pip install xgboost
%matplotlib inline
import math
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import sklearn
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,RepeatedKFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [ ]:
# Load the Dataset.
housing_df = pd.read_csv('data/housing.csv')

# Use .info() to show the features (i.e. columns) in your dataset along with a count and datatype.
housing_df.info()

In [ ]:
# Use .shape to understand how many observations (ie rows/records) of the dataset.
# The shape of a DataFrame is a tuple of array dimensions that tells the number of rows and columns of a given DataFrame.
# (row count, column count).
housing_df.shape

In [ ]:
# Using .head() function to view the first few observations (i.e. rows/records) of the dataset.
housing_df.head()

In [ ]:
# Using .tail() function to view the last few observations (i.e. records) of the dataset.
housing_df.tail()

In [ ]:
# Use .describe() to see metrics about the dataset (count, mean, std, min, 25%, 50%, 75%, max).
# Count is the count of non-null observations (i.e. rows).
# Mean is the average of values for the given column.
# Std is standard deviation - how far off from the mean the values are.
# Min is the minimum amount of the value.
# 25% is the 25th percentile.
# 50% is the 50th percentile.
# 75% is the 75th percentile.
# max is the maximum amount of the value.
housing_df.describe()

# Prepare & Preprocess Data

## Understand Missing Data

In [ ]:
# Check for missing values.
housing_df.isnull().sum()

In [ ]:
# Calculate the % of missing data.
housing_df['total_bedrooms'].isnull().sum()/housing_df.shape[0] * 100

## Use Imputation to Handle Missing Data

In [ ]:
# create a temporary copy of the dataset.
housing_df_temp = housing_df.copy()

# retrieve columns with numerical data; will exclude the ocean_proximity column since the datatype is object; other columns are float64.
columns_list = [col for col in housing_df_temp.columns if housing_df_temp[col].dtype != 'object']

# extract columns that contain at least one missing value.
new_column_list = [col for col in housing_df_temp.loc[:, housing_df_temp.isnull().any()]]

# update temp dataframe with numeric columns that have empty values.
housing_df_temp = housing_df_temp[new_column_list]

## Impute Missing Data Using Machine Learning

In [ ]:
# initialize KNNImputer to impute missing data using machine learning.
knn = KNNImputer(n_neighbors = 3)

# fit function trains the model.
knn.fit(housing_df_temp)

# transform the data using the model then apply the transformation model (ie knn) to the data.
array_Values = knn.transform(housing_df_temp)

# convert the array values to a dataframe with the appropriate column names.
housing_df_temp = pd.DataFrame(array_Values, columns = new_column_list)

In [ ]:
# confirm there are no columns with missing values.
housing_df_temp.isnull().sum()

In [ ]:
# overlay the imputed column over the old column with missing values then loop through the list of columns and overlay each one.
for column_name in new_column_list:
    housing_df[column_name] = housing_df_temp.replace(housing_df[column_name],housing_df[column_name])

# confirm columns no longer contain null data.
housing_df.isnull().sum()

# Understand the Relationship: Target Variable and Features

## Histograms

In [ ]:
# Plot the distribution of the target variable (median_house_value) using a histogram.

# bins->amount of columns.
plt.hist(housing_df['median_house_value'], bins=80)
plt.xlabel("House Values")

# We expect the plot to show Median House Values are distributed normally with some outliers. 
# Observe homes fall within th $100,000-$200,000 USD range.

In [ ]:
# Consider histograms for the all features so that we can understand data distribution.
# Use housing_df as we do not want to plot the String Datatype, OCEAN_PROXIMITY. This will be encoded to a Numeric datatype later.   
housing_df.hist(bins=50, figsize=(20,15))

## Use a heatmap to show correlation

In [ ]:
# Plot a graphical correlation matrix for each pair of columns in the dataframe.
# Below is the data frame correlation function.
corr = housing_df.corr(numeric_only=True)
print(corr)

In [ ]:
# Create the heatmap with a set size and print to screen.
plt.figure(figsize = (8,8))
sns.heatmap(corr, annot=True)
plt.show()

## Feature Engineering

In [ ]:
# Given we have very high correlation among some features, we need to manipulate data.
# Observe from the heatmap that total_rooms, total_bedrooms, population, households have high correlation.
# Could there be a change in model performance if we create ratio's of these features?

#  New Ratio: total rooms to households.
housing_df['rooms_per_household'] = housing_df['total_rooms']/housing_df['households']

# New Ratio: total bedrooms to the total rooms.
housing_df['bedrooms_per_room'] = housing_df['total_bedrooms']/housing_df['total_rooms']

# New Ratio: population to the households.
housing_df['population_per_household']= housing_df['population']/housing_df['households']

# New Ratio: latitude and longitude.
housing_df['coords'] = housing_df['longitude']/housing_df['latitude']

housing_df.info()

In [ ]:
# Remove total_rooms, households, total bedrooms, popluation, longitude, latitude as these now have a ratio.
housing_df = housing_df.drop('total_rooms', axis=1)
housing_df = housing_df.drop('households', axis=1)
housing_df = housing_df.drop('total_bedrooms', axis=1)
housing_df = housing_df.drop('population', axis=1)
housing_df = housing_df.drop('longitude', axis=1)
housing_df = housing_df.drop('latitude', axis=1)

housing_df.info()

## Heatmap after removing correlation

In [ ]:
corr = housing_df.corr(numeric_only=True) 

# Increase the size of the heatmap.
plt.figure(figsize = (7,7))

sns.heatmap(corr, annot=True)
plt.show()

# String Datatype to Numeric Datatype

In [ ]:
# Note ocean_proximity is the only categorical data type.
housing_df.info()

In [ ]:
# What unique categories exist for ocean_proximity?
housing_df.ocean_proximity.unique()

In [ ]:
# How many entries exist per category?
housing_df["ocean_proximity"].value_counts()

## One-Hot Encoding

In [ ]:
# Let's see how the Panda's get_dummies() function works. If a cell contains data, then a True is returned, otherwise False.
print(pd.get_dummies(housing_df['ocean_proximity']))

In [ ]:
# Replace the ocean_proximity column using get_dummies(), then print to screen. Note ocean_proximity column is gone.
housing_df_encoded = pd.get_dummies(data=housing_df, columns=['ocean_proximity'])
housing_df_encoded.head()

# Prepare for Model Training

In [ ]:
# remove spaces from column names, convert cell data to lowercase, and remove all special characters.
housing_df_encoded.columns = [c.lower().replace(' ', '_').replace('<', '_') for c in housing_df_encoded.columns]

# Split target variable and feature variables
X = housing_df_encoded[['housing_median_age', 'median_income','bedrooms_per_room','population_per_household','coords','ocean_proximity__1h_ocean',
                        'ocean_proximity_inland','ocean_proximity_island','ocean_proximity_near_bay','ocean_proximity_near_ocean']]
y = housing_df_encoded['median_house_value']

print(X)

## Split Training and Test Data

In [ ]:
# Using Numpy arrarys, split the data into Training (70%) and Testing (30%) sets.
# X -> array with the inputs; y -> array of the outputs
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, shuffle=True, test_size=0.3)

# Confirm how the data was split
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# Model Training

## Linear Regression 

### Create, Train, and Run

In [ ]:
# Scikit-learn’s LinearRegression will train the model on both the training dataset, then 
# evaluate on the test dataset.

# Create a Linear regressor using all the feature variables.
reg_model = LinearRegression()

# Train the model using the training sets.
reg_model.fit(X_train, y_train)

# Run the predictions on the training and testing data.
y_pred_test = reg_model.predict(X_test)

### Evaluate the model

In [ ]:
# Recall 'medianHouseValue' is the target. Note the Percent Difference between the two values. Are these predicted values acceptable to you? 
pred_test_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_test, '% Difference': abs((y_test - y_pred_test) / ((y_test + y_pred_test)) / 2 * 100)})

pred_test_df

### Accuracy and Error

In [ ]:
# Let's continue model evaluation by using the 𝑅^2 Metric. A score between 0 and 1 will represent how well the model is performing.
r2_reg_model_test = round(reg_model.score(X_test, y_test),2)

print("R^2 Test: {}".format(r2_reg_model_test))

# Determine the Root Mean Squared Error (RMSE) on the test data.
print('RMSE on test data: ',  mean_squared_error(y_test, r2_reg_model_test)**(0.5))

## RandomForestRegressor

### Create, Train, and Run

In [ ]:
# Let's use another Machine Learning algorithm, Random Forest. Scikit-learn’s RandomForestRegressor will train the 
# model on both training sets, and then evaluate using test sets.

# Create a regressor using all the feature variables avaible in the training sets. 
rf_model = RandomForestRegressor(n_estimators=10,random_state=10)

# Train the model using the training sets.
rf_model.fit(X_train, y_train)

# Run the predictions on the training and testing data for the RandomForestRegressor algorithm.
y_rf_pred_test = rf_model.predict(X_test)

### Evaluate the Model

In [ ]:
# Recall 'medianHouseValue' is the target. Note the Percent Difference between the two values. Are these predicted values acceptable to you? 
rf_pred_test_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_rf_pred_test, '% Difference': abs((y_test - y_rf_pred_test) / ((y_test + y_rf_pred_test)) / 2 * 100)})

rf_pred_test_df

### Accuracy and Error

In [ ]:
# Determine accuracy uisng the 𝑅^2 metric.
score = r2_score(y_test, y_rf_pred_test)

print("R^2 - {}%".format(round(score, 2) *100))

# Determine the Root Mean Squared Error (RMSE) on the test data.
print('RMSE on test data: ',  mean_squared_error(y_test, y_rf_pred_test)**(0.5))

### Feature Importance

In [ ]:
# The RandomForestRegressor package has an additional option for determining feature importance for all dataset varibles.
# The 6 most imporant features will be plotted.
plt.figure(figsize=(10,6))
feat_importances = pd.Series(rf_model.feature_importances_, index = X_train.columns)
feat_importances.nlargest(6).plot(kind='barh');

#### Feature Selection

In [ ]:
# Now that we have determined feature importance, let's narrow to 6 features of the orginal 10 features for learning.
# Note that we are not creating any new ratio's at this moment.
train_x_if = X_train[['bedrooms_per_room', 'housing_median_age', 'coords', 'ocean_proximity_inland','population_per_household','median_income']]
test_x_if = X_test[['bedrooms_per_room', 'housing_median_age', 'coords', 'ocean_proximity_inland','population_per_household','median_income']]

# Create an object of the RandfomForestRegressor Model.
rf_model_if = RandomForestRegressor(n_estimators=10,random_state=10)

# Fit the model with the training dataset.
rf_model_if.fit(train_x_if, y_train)

# Predict the target on the test data.
predict_test_with_if = rf_model_if.predict(test_x_if)

#### Recalculate RMSE

In [ ]:
# Determine the Root Mean Squared Error (RMSE) on the test data.
print('RMSE on test data: ',  mean_squared_error(y_test, predict_test_with_if)**(0.5))

## XGBoost

### Create, Train and Run

In [ ]:
# Extreme Gradient Boosting (XGBoost) is an open-source library that provides 
# an efficient and effective implementation of the gradient boosting algorithm.
# Use the scikit-learn wrapper classes: XGBRegressor and XGBClassifier. <----WHY?

# Create the varible representing the XGBoost algorithm.
xgb_model = XGBRegressor()

# Train the model using the Training datasets.
xgb_model.fit(X_train, y_train)

# Run the predictions on the Training and Testing datasets.
y_xgb_pred_test = xgb_model.predict(X_test)

### Evaluate the Model

In [ ]:
# Recall 'medianHouseValue' is the target. Note the Percent Difference between the two values. Are these predicted values acceptable to you? 
xgb_pred_test_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_xgb_pred_test, '% Difference': abs((y_test - y_xgb_pred_test) / ((y_test + y_xgb_pred_test)) / 2 * 100)})

xgb_pred_test_df

In [ ]:
fig= plt.figure(figsize=(8,8))
xgb_pred_test_df = xgb_pred_test_df.reset_index()
xgb_pred_test_df = xgb_pred_test_df.drop(['index'],axis=1)
plt.plot(xgb_pred_test_df[:50])
plt.legend(['Actual value','Predicted value'])

### Accuracy and Error

In [ ]:
# Determine accuracy uisng the 𝑅^2 metric.
score = r2_score(y_test, y_xgb_pred_test)

print("R^2 - {}%".format(round(score, 2) *100))

# Determine mean square error and root mean square error then print to screen.
mse = mean_squared_error(y_test, y_xgb_pred_test)
rmse = math.sqrt(mean_squared_error(y_test, y_xgb_pred_test))

print(mse)
print(rmse)

# Calculate Mean Absolute Error
print(mean_absolute_error(y_test, y_xgb_pred_test))

### Hyperparameter Tuning

In [ ]:
# Determine what Hyperparameters are available for tuning.
xgb_model.get_params()

In [ ]:
# These Hyperparameters will take on new values. Reminder, this is XGBoost specific, check varible names.
xgb_model_2 = XGBRegressor(
    gamma=0.05,
    learning_rate=0.01,
    max_depth=6,
    n_estimators=1000,
    n_jobs=16,
    objective='reg:squarederror',
    subsample=0.8,
    scale_pos_weight=0,
    reg_alpha=0,
    reg_lambda=1,
    verbosity=1)

xgb_model_2.fit(X_train, y_train)

In [ ]:
# Let's rerun the newly tuned hyperparameters using the same training and test datasets.

y_xgb_2_pred_test = xgb_model_2.predict(X_test)

In [ ]:
# Compare the actual values (ie, target) with the values predicted by the model
xgb_2_pred_test_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_xgb_2_pred_test})
xgb_2_pred_test_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_xgb_2_pred_test, '% Difference': abs((y_test - y_xgb_2_pred_test) / ((y_test + y_xgb_2_pred_test)) / 2 * 100)})

xgb_2_pred_test_df

In [ ]:
# Let's vizualize actual values and predicted values within the XGBoost.
fig= plt.figure(figsize=(8,8))
xgb_2_pred_test_df = xgb_2_pred_test_df.reset_index()
xgb_2_pred_test_df = xgb_2_pred_test_df.drop(['index'],axis=1)
plt.plot(xgb_2_pred_test_df[:50])
plt.legend(['Actual value','Predicted value'])

#### Post-Tuning: Accuracy and Error 

In [ ]:
# Determine accuracy uisng 𝑅^2.
r2_xgb_model_2_test = round(xgb_model_2.score(X_test, y_test),2)

print("R^2 Test: {}".format(r2_xgb_model_2_test))

# Determine mean square error and root mean square error then print to screen.
mse = mean_squared_error(y_test, y_xgb_2_pred_test)
rmse = math.sqrt(mean_squared_error(y_test, y_xgb_2_pred_test))

print(mse)
print(rmse)

# Cross Validation

In [ ]:
# We can build and score a model on multiple folds using cross validation.

# Define a model evaluation method.
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

scores = cross_val_score(xgb_model, X, y, scoring='r2', error_score='raise', cv=cv, n_jobs=-1, verbose=1)

# Average of all the r2 scores across all runs.
print(scores.mean())